In [5]:
import pandas as pd
import numpy as np

In [1]:
def loadData(flieName):  
    inFile = open(flieName, 'r')
    x = []  
    y = []    
    z = [] 
    t = []
    for line in inFile:  
        trainingSet = line.split('::')  
        x.append(int(trainingSet[0]))
        y.append(int(trainingSet[1])) 
        z.append(float(trainingSet[2])) 
        t.append(trainingSet[3])

    return (x, y, z, t)

In [14]:
ratingsFrame = pd.DataFrame(np.array(loadData("ml-1m/ratings.dat"))).T.loc[:,0:2]
# ratingsFrame.columns = ['userID','itemID','rating']
ratingsFrame[0] = ratingsFrame[0].astype('int')
ratingsFrame[1] = ratingsFrame[1].astype('int')
ratingsFrame[2] = ratingsFrame[2].astype('float')

In [3]:
catagory = "Cell_Phones_and_Accessories"
item_data = 'item_metadata/meta_' + catagory + '.csv'
rating_data = 'user_ratings/' + catagory + '.csv'

In [2]:
ratingsFrame = pd.read_csv(rating_data)
itemsFrame = pd.read_csv(item_data)

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Load a text file and convert each line to a Row.
ratings = ratingsFrame.to_records(index=False).tolist()
dataset = spark.createDataFrame(ratings,["userId", "ProductId", "rating"])
# lines = spark.read.text(rating_data).rdd
# parts = lines.map(lambda row: row.value.split(","))
# parts.count()

In [5]:
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), ProductId=int(p[1]),
#                                      rating=float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])
test.head(3)

[Row(ProductId=0, rating=5.0, userId=20),
 Row(ProductId=1, rating=5.0, userId=62),
 Row(ProductId=1, rating=5.0, userId=65)]

In [6]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="ProductId", ratingCol="rating",
          coldStartStrategy="drop")

In [7]:
model = als.fit(training)

In [8]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [9]:
rmse = evaluator.evaluate(predictions)

In [10]:
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 1.4078966863240838


In [ ]:
Cell_Phones_and_Accessories : 1.1990174729619572
    users:    90.0
    items:    115.0
    ratings:    1854
    ratio:    0.179130434783
Grocery_and_Gourmet_Food : 1.5124280016578064
    users:    1748.0
    items:    548.0
    ratings:    37348
    ratio:    0.0389892932904
CDs_and_Vinyl: 1.9617218828545564
    users:    15591.0
    items:    16183.0
    ratings:    445412
    ratio:    0.00176534221888
Movies_and_TV : 1.7274201662946715
    users:    33325.0
    items:    21900.0
    ratings:    958986
    ratio:    0.00131400795404

In [ ]:
Video_Games : 3.9319703276510016
Electronics: 5.129375041478559
Kindle_Store : 2.9681531710437223
Books : 2.866274719315073

In [ ]:
Video_Games:
    users:    2815.0
    items:    2140.0
    ratings:    52158
    ratio:    0.00865822280507
Electronics:
    users:    20246.0
    items:    11588.0
    ratings:    347393
    ratio:    0.00148072136948
Kindle_Store:
    users:    14355.0
    items:    15884.0
    ratings:    367478
    ratio:    0.00161164085738
Books:
    users:    158649.0
    items:    128938.0
    ratings:    4701968
    ratio:    0.000229858940579